In [5]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [6]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_2_Choosing_Investment_Strategies/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-30 12:56:05.213706


## Step 1 - Generate Mathematical Formulation 

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

To formulate a mean-variance mathematical optimization model for this problem, we'll define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- Let \(N = 10\) be the number of investment strategies.
- Let \(A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]\) be the vector of investment costs.
- Let \(R = [r_{i,j}]\) be the \(N \times M\) matrix of historic returns, where \(r_{i,j}\) is the return of strategy \(i\) in year \(j\), and \(M\) is the number of rows in the "investments_data.csv" file.
- Let \(B = [1000, 1100, 900, 1200]\) be the vector of possible budget scenarios.
- Let \(P = [0.55, 0.4, 0.04, 0.01]\) be the vector of probabilities for each budget scenario.
- Let \(r = 2\) be the risk parameter.

Decision Variables:
- Let \(x_i\) be a binary decision variable indicating whether we invest in strategy \(i\) (\(x_i = 1\)) or not (\(x_i = 0\)).
- Let \(z\) be a continuous decision variable representing the expected return on investment.
- Let \(w\) be a continuous decision variable representing the variance of the return on investment.

Objective Function:
We aim to maximize the expected return \(z\) while minimizing the risk \(r \cdot w\). Thus, the objective function is:
\[ \text{maximize } z - r \cdot w \]

Constraints:
1. Budget constraint:
For each budget scenario \(k\), the total investment cost should not exceed the budget. To account for the tolerable probability of exceeding the budget, we use the Value-at-Risk (VaR) approach. The constraint is:
\[ \text{VaR}_{0.4}(B) \geq \sum_{i=1}^{N} A_i \cdot x_i \]

2. Expected return calculation:
The expected return \(z\) is calculated as the weighted average of historic returns, where the weights are the decision variables \(x_i\):
\[ z = \frac{1}{M} \sum_{i=1}^{N} \sum_{j=1}^{M} r_{i,j} \cdot x_i \]

3. Risk calculation:
The risk \(w\) is calculated as the variance of the historic returns, weighted by the decision variables \(x_i\):
\[ w = \frac{1}{M} \sum_{i=1}^{N} \sum_{j=1}^{M} (r_{i,j} - z)^2 \cdot x_i \]

4. Binary constraints:
\[ x_i \in \{0, 1\} \quad \forall i \in \{1, 2, \dots, N\} \]

The resulting optimization problem is a mixed-integer quadratic program that can be solved using a suitable optimization solver.

In [7]:
print(response_1_text)

To formulate a mean-variance mathematical optimization model for this problem, we'll define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- Let \(N = 10\) be the number of investment strategies.
- Let \(A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]\) be the vector of investment costs.
- Let \(R = [r_{i,j}]\) be the \(N \times M\) matrix of historic returns, where \(r_{i,j}\) is the return of strategy \(i\) in year \(j\), and \(M\) is the number of rows in the "investments_data.csv" file.
- Let \(B = [1000, 1100, 900, 1200]\) be the vector of possible budget scenarios.
- Let \(P = [0.55, 0.4, 0.04, 0.01]\) be the vector of probabilities for each budget scenario.
- Let \(r = 2\) be the risk parameter.

Decision Variables:
- Let \(x_i\) be a binary decision variable indicating whether we invest in strategy \(i\) (\(x_i = 1\)) or not (\(x_i = 0\)).
- Let \(z\) be a continuous decision variable representing the expected return on inve

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

Here is a Python code using Pyomo to solve the mean-variance optimization problem:

```python
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
P = np.array([0.55, 0.4, 0.04, 0.01])
r = 2
N = len(A)
M = 10  # Sample data for M, replace with actual value from investments_data.csv

# Read historic returns data
investments_data = pd.read_csv('investments_data.csv')
R = investments_data.values

# Pyomo model
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(range(N), domain=pyo.Binary)
model.z = pyo.Var()
model.w = pyo.Var()

# Objective function
model.obj = pyo.Objective(expr=model.z - r * model.w, sense=pyo.maximize)

# Constraints
def budget_constraint_rule(model):
    return sum(A[i] * model.x[i] for i in range(N)) <= np.percentile(B, 40)
model.budget_constraint = pyo.Constraint(rule=budget_constraint_rule)

def expected_return_rule(model):
    return model.z == (1 / M) * sum(sum(R[i, j] * model.x[i] for j in range(M)) for i in range(N))
model.expected_return_constraint = pyo.Constraint(rule=expected_return_rule)

def risk_rule(model):
    return model.w == (1 / M) * sum(sum((R[i, j] - model.z)**2 * model.x[i] for j in range(M)) for i in range(N))
model.risk_constraint = pyo.Constraint(rule=risk_rule)

# Solver
solver = pyo.SolverFactory('ipopt')

# Solve the problem
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("x =", [pyo.value(model.x[i]) for i in range(N)])
print("Expected return (z) =", pyo.value(model.z))
print("Risk (w) =", pyo.value(model.w))
```

You need to replace the sample data for `M` with the actual number of rows in your `investments_data.csv` file. Also, make sure to install the required packages (Pyomo, NumPy, and Pandas) and the `ipopt` solver before running the code.

In [11]:
print(response_2_text)

Here is a Python code using Pyomo to solve the mean-variance optimization problem:

```python
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
P = np.array([0.55, 0.4, 0.04, 0.01])
r = 2
N = len(A)
M = 10  # Sample data for M, replace with actual value from investments_data.csv

# Read historic returns data
investments_data = pd.read_csv('investments_data.csv')
R = investments_data.values

# Pyomo model
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(range(N), domain=pyo.Binary)
model.z = pyo.Var()
model.w = pyo.Var()

# Objective function
model.obj = pyo.Objective(expr=model.z - r * model.w, sense=pyo.maximize)

# Constraints
def budget_constraint_rule(model):
    return sum(A[i] * model.x[i] for i in range(N)) <= np.percentile(B, 40)
model.budget_constraint = pyo.Constraint(rule=budget_constraint_rule)

def expected_return_rule(model):
    re

### Code Executability

In [7]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
P = np.array([0.55, 0.4, 0.04, 0.01])
r = 2
N = len(A)
M = 10  # Sample data for M, replace with actual value from investments_data.csv

# Read historic returns data
investments_data = pd.read_csv(FILE_PATH_PROBLEM + 'investments_data.csv')
R = investments_data.values

# Pyomo model
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(range(N), domain=pyo.Binary)
model.z = pyo.Var()
model.w = pyo.Var()

# Objective function
model.obj = pyo.Objective(expr=model.z - r * model.w, sense=pyo.maximize)

# Constraints
def budget_constraint_rule(model):
    return sum(A[i] * model.x[i] for i in range(N)) <= np.percentile(B, 40)
model.budget_constraint = pyo.Constraint(rule=budget_constraint_rule)

def expected_return_rule(model):
    return model.z == (1 / M) * sum(sum(R[i, j] * model.x[i] for j in range(M)) for i in range(N))
model.expected_return_constraint = pyo.Constraint(rule=expected_return_rule)

def risk_rule(model):
    return model.w == (1 / M) * sum(sum((R[i, j] - model.z)**2 * model.x[i] for j in range(M)) for i in range(N))
model.risk_constraint = pyo.Constraint(rule=risk_rule)

# Solver
solver = pyo.SolverFactory('gurobi')

# Solve the problem
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("x =", [pyo.value(model.x[i]) for i in range(N)])
print("Expected return (z) =", pyo.value(model.z))
print("Risk (w) =", pyo.value(model.w))

2024-05-30 12:56:25,262 ERROR pyomo.core: Rule failed when generating expression for Constraint expected_return_constraint with index None:
AttributeError: 'str' object has no attribute 'is_potentially_variable'
2024-05-30 12:56:25,264 ERROR pyomo.core: Constructing component 'expected_return_constraint' from data=None failed:
AttributeError: 'str' object has no attribute 'is_potentially_variable'


AttributeError: 'str' object has no attribute 'is_potentially_variable'

### Solution Correctness